Using the DHAS fits files that MAGIC creates, calculate what the expected count rates that DHAS will measure will be. 

In [1]:
import os

from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np

from jwst_magic.utils import utils
from jwst_magic.fsw_file_writer import buildfgssteps

Using backend:  Qt5Agg
Your MAGIC package is up to date
Your FGS COUNTRATE package is up to date


#### Functions 

In [2]:
def create_merged_id_full_frame(strips, nstrips=36, strip_height=64, yoffset=12, overlap=8, 
                                image_size=2048):
    """
    Put an array of strips of size (nstrips, strip_height, image_size) into an array that 
    is image_size by image_size, accounting for the overlap
    
    """
    full_frame_merged = np.zeros((image_size, image_size))
    
    for i, strip in enumerate(strips):
        if i == 0:
            full_frame_merged[yoffset:yoffset+strip_height] = strip
        else:
            low = i * (strip_height - overlap) + yoffset
            full_frame_merged[low + overlap :low + strip_height]=strip[overlap:]

    return full_frame_merged

In [3]:
def estimate_measured_count_rate(data, tcds):
    '''
    Data is the ID, ACQ1, or ACQ2 data created by MAGIC for the DHAS in units of counts

    '''
    box_size = np.shape(data[0])[1]
    gs_y = box_size//2 #- .5
    gs_x = box_size//2 #- .5

    data_counts = buildfgssteps.create_cds(data) # TODO: only do this if it hasn't already been done at this point
    # Only for ID, make the merged full frame images
    if np.shape(data_counts) == (72, 64, 2048):
        id_full_frame_1 = create_merged_id_full_frame(data_counts[::2])
        id_full_frame_2 = create_merged_id_full_frame(data_counts[1::2])
        data_counts = [id_full_frame_1, id_full_frame_2]
    
    median_counts = np.median(data_counts, axis=0)

    # getting counts even though function says count rate
    counts_sum = utils.get_countrate_3x3(gs_x, gs_y, median_counts) 

    # extrapolate the 3x3 CR associated with the summed counts
    esimated_count_rate = counts_sum/tcds
    
    return esimated_count_rate

#### Constants

In [4]:
# CDS times for ID, ACQ1, and ACQ2
tcds_id = 0.3406
tcds_acq1 = 0.3612
tcds_acq2 = 0.05016

## Test on actual data

In [5]:
# If you want to run this on your own data, replace the below variables to match your data
root = 'test_unlimited_cr_7mag'
out_dir = '/Users/kbrooks/git_repos/jwst-magic-fork/out'
guider = 1 
config = 1

In [6]:
# Build filenames matching MAGIC naming structure 
id_data = fits.getdata(os.path.join(out_dir, root, f"guiding_config_{config}",
                                    "dhas_shifted", f"{root}_G{guider}_IDstrips.fits"))

acq1_data = fits.getdata(os.path.join(out_dir, root, f"guiding_config_{config}",
                                      "dhas_shifted", f"{root}_G{guider}_ACQ1.fits"))

acq2_data = fits.getdata(os.path.join(out_dir, root, f"guiding_config_{config}",
                                      "dhas_shifted", f"{root}_G{guider}_ACQ2.fits"))

In [7]:
# Look at the ACQ1 data, reads 1 and 2 - this data is very saturated
fig, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].imshow(acq1_data[0])
ax[0].set_title("Read 1")

ax[1].imshow(acq1_data[1])
ax[1].set_title("Read 2")

Text(0.5, 1.0, 'Read 2')

## ACQ1

In [8]:
estimated_acq1_count_rate = estimate_measured_count_rate(acq1_data, tcds_acq1)
estimated_acq1_count_rate

622923.5880398671

## ACQ2

In [9]:
estimated_acq2_count_rate = estimate_measured_count_rate(acq2_data, tcds_acq2)
estimated_acq2_count_rate

2396172.2488038274

In [10]:
# Failure will be unavoidable when 2 * ACQ1 CR > ACQ2 CR
failure = 2 * estimated_acq1_count_rate < estimated_acq2_count_rate
print(f"Is it possible to avoid failure with this data? {'no' if failure else 'yes'}")

Is it possible to avoid failure with this data? no


## ID 

In [11]:
estimated_id_count_rate = estimate_measured_count_rate(id_data, tcds_id)
estimated_id_count_rate

660598.9430416911

## Compare the above values with "truth" (what DHAS spits out) 

In [12]:
# "Truth" values - the values returned by DHAS
dhas_id_cr = 660598.943 #saturated
dhas_acq1_cr = 622923.588 #saturated
dhas_acq2_crs = [2380303, 2375837, 2468919, 2405083, 2385785]
dhas_acq2_cr = np.median(dhas_acq2_crs)
dhas_acq2_cr_std = np.std(dhas_acq2_crs)


In [13]:
print('Approximately what I am aiming for')
print(f'\t ID CR: {dhas_id_cr}')
print(f'\t ACQ1 CR: {dhas_acq1_cr}')
print(f'\t ACQ2 CR: {dhas_acq2_cr} +/- {dhas_acq2_cr_std}')
print(f'\t (ACQ2 CR range: {np.round(dhas_acq2_cr-dhas_acq2_cr_std)} - {np.round(dhas_acq2_cr+dhas_acq2_cr_std)})')

Approximately what I am aiming for
	 ID CR: 660598.943
	 ACQ1 CR: 622923.588
	 ACQ2 CR: 2385785.0 +/- 34347.693335069824
	 (ACQ2 CR range: 2351437.0 - 2420133.0)


### About this notebook
Author: K. Brooks

Last updated: 6 October, 2021